# Limpieza de Datos

In [1]:
import pandas as pd

## Análisis Exploratorio de Datos

Como resultado del [Análisis Exploratorio de Datos](notebooks/data_exploration.ipynb) se han identificado algunas variables que no seran consideradas.

In [5]:
df = pd.read_csv("sba_national.csv", low_memory=False)
drop_cols = ["LoanNr_ChkDgt", "Name", "City", "Zip", "Bank", "ApprovalFY", "FranchiseCode"]
df = df.drop(drop_cols, axis=1)

print(df.shape)
df.sample(3)

(899164, 20)


,State,BankState,NAICS,ApprovalDate,Term,NoEmp,NewExist,CreateJob,RetainedJob,UrbanRural,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
440694,TX,OK,421110,16-Oct-90,120,45,1.0,0,0,0,N,N,NaN,31-Oct-90,"$750,000.00",$0.00,P I F,$0.00,"$750,000.00","$487,500.00"
207128,LA,LA,722211,6-Oct-06,156,1,2.0,10,0,0,0,N,NaN,30-Apr-07,"$150,000.00",$0.00,P I F,$0.00,"$150,000.00","$127,500.00"
32063,IN,DE,453998,23-Mar-05,240,1,1.0,2,1,2,N,N,NaN,31-May-05,"$100,000.00",$0.00,P I F,$0.00,"$100,000.00","$50,000.00"


In [6]:
df.dtypes

State                 object
BankState             object
NAICS                  int64
ApprovalDate          object
Term                   int64
NoEmp                  int64
NewExist             float64
CreateJob              int64
RetainedJob            int64
UrbanRural             int64
RevLineCr             object
LowDoc                object
ChgOffDate            object
DisbursementDate      object
DisbursementGross     object
BalanceGross          object
MIS_Status            object
ChgOffPrinGr          object
GrAppv                object
SBA_Appv              object
dtype: object

In [8]:
df.isnull().sum()

State                    14
BankState              1566
NAICS                     0
ApprovalDate              0
Term                      0
NoEmp                     0
NewExist                136
CreateJob                 0
RetainedJob               0
UrbanRural                0
RevLineCr              4528
LowDoc                 2582
ChgOffDate           736465
DisbursementDate       2368
DisbursementGross         0
BalanceGross              0
MIS_Status             1997
ChgOffPrinGr              0
GrAppv                    0
SBA_Appv                  0
dtype: int64

## Validación de Datos

In [ ]:
# drop duplicates

## Imputación de Datos

## Detección de Outliers

## Conclusiones